In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                                  options=[ 
												       {'label': 'ALL', 'value': 'ALL'},
                                                       {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                       {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                       {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                       {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                          ],
                                                        value='ALL',
                                                         placeholder="Select a launch site",
                                                         searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min = 0, max = 10000, step = 1000,
                                                marks = {0:'0', 1000:'1000',2000:'2000',3000:'3000',4000:'4000',5000:'5000',6000:'6000',7000:'7000',8000:'8000',9000:'9000',10000:'10000'},
                                                value = [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))

def get_pie_chart(entered_site):

    if entered_site == 'ALL':
        data = spacex_df
        fig_all = px.pie(data , values ='class', names = 'Launch Site' ,title = 'Total success launches for ALL sites')
        return fig_all
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        data1 = filtered_df.groupby(['class']).size().reset_index(name='class count')
        fig_site = px.pie(data1, names ='class', values = 'class count' ,title = 'Total success launches for {} sites'.format(entered_site))
        return fig_site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")]
             )
def get_scatter_chart(entered_site, payload_value):
    filterd_df  = spacex_df[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        fig_scatter_all = px.scatter(spacex_df, x="Payload Mass (kg)", y="class",color="Booster Version Category",title ="Correlation between Payload and Success for all sites")
        return fig_scatter_all
    else:
        fig_scatter_site = px.scatter(filterd_df, x="Payload Mass (kg)", y="class",color="Booster Version Category",title ="Correlation between Payload and Success for {} site".format(entered_site))
        return fig_scatter_site
# Run the app
if __name__ == '__main__':
    run_server(app, "127.0.0.1",8000)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_0m1632320853.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_0m1632320853.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_0m1632320853.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_15_0m1632320857.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_0m1632320853.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_15_0m1632320857.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Python39\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Python39\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Python39\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Python39\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Python39\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Python39\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Python39\lib\site-packages\dash\dash.py", line 1078, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Python39\lib\site-packages\dash\dash

127.0.0.1 - - [02/Mar/2022 12:25:14] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Mar/2022 12:25:14] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_15_0m1611086576.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:25:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:28:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:28:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:31:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:31:02] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [02/Mar/2022 12:31:02] "GET /_dash-layout